In [1]:
import pandas as pd
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("Data/Final_data.csv")
df.head()

,Unnamed: 0_x,ArrDel15,ArrDelayMinutes,ArrTime,CRSArrTime,CRSDepTime,DayofMonth,DepDel15,DepDelayMinutes,DepTime,...,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y,time_y,date_y,airport_y
0,0,0.0,8.0,1610.0,1602,745,1,0.0,0.0,741.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
1,557,1.0,25.0,1628.0,1603,1330,1,1.0,38.0,1408.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
2,282,0.0,0.0,1540.0,1600,1310,1,0.0,0.0,1306.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
3,51602,0.0,0.0,1508.0,1530,700,1,0.0,0.0,659.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK
4,1506,0.0,0.0,1600.0,1624,800,1,0.0,0.0,758.0,...,1015,0,32,26,41,32,74,1600,2016-01-01,JFK


In [3]:
df = df.drop(columns=['Unnamed: 0_x', 'time_x', 'date_x','time_y', 'date_y', 'FlightDate', 'Unnamed: 0_y', 'airport_y', 
                      'airport_x', 'ArrTime', 'Time_new_dept', 'Time_new_arr','Unnamed: 0'])
df.head()

,ArrDel15,ArrDelayMinutes,CRSArrTime,CRSDepTime,DayofMonth,DepDel15,DepDelayMinutes,DepTime,DestAirportID,Month,...,weatherCode_y,precipMM_y,visibility_y,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y
0,0.0,8.0,1602,745,1,0.0,0.0,741.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
1,1.0,25.0,1603,1330,1,1.0,38.0,1408.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
2,0.0,0.0,1600,1310,1,0.0,0.0,1306.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
3,0.0,0.0,1530,700,1,0.0,0.0,659.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74
4,0.0,0.0,1624,800,1,0.0,0.0,758.0,12478,1,...,113,0.0,10,1015,0,32,26,41,32,74


In [4]:
df.columns

Index(['ArrDel15', 'ArrDelayMinutes', 'CRSArrTime', 'CRSDepTime', 'DayofMonth',
       'DepDel15', 'DepDelayMinutes', 'DepTime', 'DestAirportID', 'Month',
       'Origin', 'OriginAirportID', 'Quarter', 'Year', 'Dest',
       'windspeedKmph_x', 'winddirDegree_x', 'weatherCode_x', 'precipMM_x',
       'visibility_x', 'pressure_x', 'cloudcover_x', 'DewPointF_x',
       'WindGustKmph_x', 'tempF_x', 'WindChillF_x', 'humidity_x',
       'windspeedKmph_y', 'winddirDegree_y', 'weatherCode_y', 'precipMM_y',
       'visibility_y', 'pressure_y', 'cloudcover_y', 'DewPointF_y',
       'WindGustKmph_y', 'tempF_y', 'WindChillF_y', 'humidity_y'],
      dtype='object')

In [5]:
from sklearn import preprocessing
label = preprocessing.LabelEncoder()
df['Origin'] = label.fit_transform(df['Origin'])
df['Dest'] = label.fit_transform(df['Dest'])

In [6]:
y = df[['ArrDel15', 'ArrDelayMinutes']]
X = df.drop(columns = ['ArrDel15', 'ArrDelayMinutes'])

In [7]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.70, random_state = 50)

In [8]:
X_train.shape

(1295527, 37)

In [9]:
from collections import Counter
Counter(y_train['ArrDel15'])

Counter({1.0: 271516, 0.0: 1024011})

In [10]:
y_train_classification, y_test_classification = y_train["ArrDel15"], y_test["ArrDel15"]
y_test_regression = y_test["ArrDelayMinutes"]

In [11]:
import xgboost
from xgboost import XGBClassifier
xgboost_model = XGBClassifier()
xbgoost_classifier = xgboost_model.fit(X_train, y_train_classification)

In [12]:
y_pred_xgboost = xgboost_model.predict(X_test)

In [13]:
print(classification_report(y_test_classification, y_pred_xgboost))

              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    438809
         1.0       0.90      0.70      0.79    116418

    accuracy                           0.92    555227
   macro avg       0.91      0.84      0.87    555227
weighted avg       0.92      0.92      0.92    555227



In [14]:
X_test["Predicted_ArrDel15"] = y_pred_xgboost
X_test = X_test.loc[:][X_test["Predicted_ArrDel15"]==1]
X_test = X_test.drop(["Predicted_ArrDel15"],axis=1)
X_test.shape

(90636, 37)

In [15]:
y_test_regression_new= pd.DataFrame(y_test_regression)
y_test_regression_new["Predicted_ArrDel15"] = y_pred_xgboost
y_test_regression_new = y_test_regression_new.loc[y_test_regression_new["Predicted_ArrDel15"] == 1]
y_test_regression_new = y_test_regression_new.drop(["Predicted_ArrDel15"], axis = 1)

In [16]:
y_test_regression_new = pd.Series(y_test_regression_new["ArrDelayMinutes"])

In [17]:
X_test.shape, y_test_regression_new.shape

((90636, 37), (90636,))

In [18]:
X_train["ArrDel15"] = y_train["ArrDel15"]
X_train = X_train.loc[X_train["ArrDel15"]==1]
X_train = X_train.drop("ArrDel15", axis = 1)
y_train_regression = y_train.loc[y_train["ArrDel15"] == 1]
y_train_regression = y_train_regression.drop("ArrDel15", axis = 1)

In [19]:
X_train.shape, y_train_regression.shape, X_test.shape, y_test_regression_new.shape

((271516, 37), (271516, 1), (90636, 37), (90636,))

In [20]:
from xgboost import XGBRegressor
xgboost_regressor = XGBRegressor()
xgboost_regressor.fit(X_train,y_train_regression)
y_pred_xgboost_regression = xgboost_regressor.predict(X_test)

In [21]:
from sklearn import metrics
import numpy as np
mean_absolute_error=metrics.mean_absolute_error(y_test_regression_new, y_pred_xgboost_regression) 
MSE=metrics.mean_squared_error(y_test_regression_new, y_pred_xgboost_regression) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test_regression_new, y_pred_xgboost_regression)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)

12.861288330130819
308.2526406549859
17.557125068045334
0.9511603905459871


In [49]:
X_test["ArrDelayMinutes"] = y_test_regression_new
X_test['ArrDelayMinutes'].max()

1649.0

### Class 15 to  100

In [23]:
X_test["ArrDelayMinutes"] = y_test_regression_new
df_temp1 = X_test.loc[X_test['ArrDelayMinutes'].isin(list(range(15,100)))]

X_test1 = df_temp1.drop(columns = ['ArrDelayMinutes'])
y_test1 = df_temp1['ArrDelayMinutes']

y_pred_xgboost1 = xgboost_regressor.predict(X_test1)

mean_absolute_error=metrics.mean_absolute_error(y_test1, y_pred_xgboost1) 
MSE=metrics.mean_squared_error(y_test1, y_pred_xgboost1) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test1, y_pred_xgboost1)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)

10.380981653334002
183.563731265749
13.548569343873508
0.6386016194874876


### Class 100 to  200

In [24]:
df_temp2 = X_test.loc[X_test['ArrDelayMinutes'].isin(list(range(100,200)))]

X_test2 = df_temp2.drop(columns = ['ArrDelayMinutes'])
y_test2 = df_temp2['ArrDelayMinutes']

y_pred_xgboost2 = xgboost_regressor.predict(X_test2)

mean_absolute_error=metrics.mean_absolute_error(y_test2, y_pred_xgboost2) 
MSE=metrics.mean_squared_error(y_test2, y_pred_xgboost2) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test2, y_pred_xgboost2)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)

15.795037762429804
537.7764154793003
23.190006802053773
0.26985265138536174


### Class 200 to  500

In [25]:
df_temp3 = X_test.loc[X_test['ArrDelayMinutes'].isin(list(range(200,500)))]

X_test3 = df_temp3.drop(columns = ['ArrDelayMinutes'])
y_test3 = df_temp3['ArrDelayMinutes']

y_pred_xgboost3 = xgboost_regressor.predict(X_test3)

mean_absolute_error=metrics.mean_absolute_error(y_test3, y_pred_xgboost3) 
MSE=metrics.mean_squared_error(y_test3, y_pred_xgboost3) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test3, y_pred_xgboost3)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)

17.77827537675076
775.9730760456407
27.856293293359055
0.8227200853290012


### Class 500 to 1000

In [26]:
df_temp4 = X_test.loc[X_test['ArrDelayMinutes'].isin(list(range(500,1000)))]

X_test4 = df_temp4.drop(columns = ['ArrDelayMinutes'])
y_test4 = df_temp4['ArrDelayMinutes']

y_pred_xgboost4 = xgboost_regressor.predict(X_test4)

mean_absolute_error=metrics.mean_absolute_error(y_test4, y_pred_xgboost4) 
MSE=metrics.mean_squared_error(y_test4, y_pred_xgboost4) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test4, y_pred_xgboost4)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)

16.448859709381715
559.9477780436654
23.663215716458854
0.9704709722532535


### Class 1000 to 1650

In [50]:
df_temp5 = X_test.loc[X_test['ArrDelayMinutes'].isin(list(range(1000,1650)))]

X_test5 = df_temp5.drop(columns = ['ArrDelayMinutes'])
y_test5 = df_temp5['ArrDelayMinutes']

y_pred_xgboost5 = xgboost_regressor.predict(X_test5)

mean_absolute_error=metrics.mean_absolute_error(y_test5, y_pred_xgboost5) 
MSE=metrics.mean_squared_error(y_test5, y_pred_xgboost5) 
RMSE= np.sqrt(MSE)
R2=metrics.r2_score(y_test5, y_pred_xgboost5)
print(mean_absolute_error)
print(MSE)
print(RMSE)
print(R2)

22.85957768278302
1060.9127986414253
32.571656369325545
0.9403339498065914
